# On the Usage of LLM

ChatGPT and other LLM are limited and can only help you so much. Overusage leads to a lack of ability to understand and comprehend your code.
But with that said, LLM are excellent debugging tools and having conversations and solutionoriented conversations with a LLM is the future, i remember the time i had to sit on stackoverflow and being stuck could result in willpower being drained as well as just ending up stuck untill you stumble upon the solution or ask someone else. 

So we love LLM and we look forward to helping them improve and improve with them, the dream programmers had of a rubberduck that can talk back and help you achieve that is already here. So during parts of hte code you will see exert of GPT documentation, that is mainly the result copy paste syntax.

In [119]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix, lil_matrix, hstack
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns
from fuzzywuzzy import process, fuzz

In [64]:
df_ratings = pd.read_csv(
    "data/ratings.csv",
    dtype={"userId": "int32", "movieId": "int32", "rating": "float32"}
)

# Calculate the reviews count and average rating, then reset index
# Ensure "reviews" is kept as an int
ratings_count = df_ratings.groupby("movieId").size().reset_index(
    name="reviews")
average_ratings = df_ratings.groupby(
    "movieId")["rating"].mean().round(1).reset_index(name="Rating")

# Read in the movies data
df_movies = pd.read_csv(
    "data/movies.csv",
    usecols=["movieId", "title", "genres"],
    dtype={"movieId": "int32", "title": "str", "genres": "str"}
)

df_movies["genres"] = df_movies["genres"].apply(
    lambda x: [genre.lower() for genre in x.split("|")])

df_merged = df_movies.merge(ratings_count, on="movieId", how="left").merge(
    average_ratings, on="movieId", how="left")

df_filtered_movies = df_merged[df_merged["reviews"] >= 300].dropna()

# Convert "reviews" to int explicitly
df_filtered_movies["reviews"] = df_filtered_movies["reviews"].astype("int32")

# Set "movieId" as the index
df_filtered_movies.set_index("movieId", inplace=True)
df_filtered_movies.reset_index(drop=True, inplace=True)

In [138]:
top_rated_movies = df_filtered_movies.sort_values(
    by="reviews", ascending=False).head(int(len(df_filtered_movies) * 0.1))

bottom_rated_movies = df_filtered_movies.sort_values(
    by="reviews", ascending=False).tail(int(len(df_filtered_movies) * 0.1))

In [49]:
df_filtered_movies.head(5)


,title,genres,reviews,Rating
0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",76813,3.9
1,Jumanji (1995),"[Adventure, Children, Fantasy]",30209,3.3
2,Grumpier Old Men (1995),"[Comedy, Romance]",15820,3.2
3,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",3028,2.9
4,Father of the Bride Part II (1995),[Comedy],15801,3.1


In [50]:
df_filtered_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7876 entries, 0 to 7875
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   title    7876 non-null   object 
 1   genres   7876 non-null   object 
 2   reviews  7876 non-null   int32  
 3   Rating   7876 non-null   float32
dtypes: float32(1), int32(1), object(2)
memory usage: 184.7+ KB


## Baysian Average

It makes the data more reliable for items with less reviews in a dataset with a mix of both. We have cut off movies with less than 300 reviews in our cold start so that might not be nessecary, but still cool to have.

In [67]:
m = df_ratings["rating"].mean()
C = df_ratings.groupby("movieId").size().mean()

#Baysian might be used later if our dataset 
df_filtered_movies["bayesian_avg"] = (
    df_filtered_movies["reviews"] * df_filtered_movies["Rating"] + C * m) / (df_filtered_movies["reviews"] + C)     #The actual algorithm
df_filtered_movies["bayesian_avg"] = df_filtered_movies["bayesian_avg"].round(1)                                    #Adding it back inside rounded to 1 deciminal


In [52]:
df_filtered_movies

,title,genres,reviews,Rating,bayesian_avg
0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",76813,3.9,3.9
1,Jumanji (1995),"[Adventure, Children, Fantasy]",30209,3.3,3.3
2,Grumpier Old Men (1995),"[Comedy, Romance]",15820,3.2,3.2
3,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",3028,2.9,3.0
4,Father of the Bride Part II (1995),[Comedy],15801,3.1,3.1
...,...,...,...,...,...
7871,John Wick: Chapter 4 (2023),"[Action, Crime, Thriller]",651,3.7,3.6
7872,Dungeons & Dragons: Honor Among Thieves (2023),"[Action, Adventure, Fantasy]",488,3.7,3.6
7873,Guardians of the Galaxy Volume 3 (2023),"[Action, Adventure, Sci-Fi]",534,3.8,3.7
7874,The Super Mario Bros. Movie (2023),"[Adventure, Animation, Children, Comedy, Fantasy]",366,3.3,3.4


# Test-data tinkering small.

In [95]:
def create_feature_matrix(df):
    """
    Given a dataframe with movies and their genres and Bayesian average ratings,
    create a feature matrix using TF-IDF for genres and scaling for Bayesian average.
    """
    def identity_tokenizer(text):
        return text

    vectorizer = TfidfVectorizer(
        tokenizer=identity_tokenizer, lowercase=False, preprocessor=lambda x: x)    #Using new tokenizer rules because it won't translate the genre data correctly.
    genres_matrix = vectorizer.fit_transform(df["genres"])

    scaler = MinMaxScaler()
    bayesian_avg_scaled = scaler.fit_transform(
        df[["bayesian_avg"]])       # Convert to CSR format


    # Convert to CSR format
    feature_matrix = hstack([genres_matrix, bayesian_avg_scaled]).tocsr()

    return feature_matrix, vectorizer, scaler, genres_matrix #Temporary Genres Matrix for EDA


feature_matrix, vectorizer, scaler, genres_matrix = create_feature_matrix(
    df_filtered_movies)

# You can now proceed to create a DataFrame for the genres_matrix to explore the TF-IDF scores
genres_tfidf_df = pd.DataFrame(
    genres_matrix.toarray(), columns=sorted(vectorizer.vocabulary_.keys()))

In [70]:
feature_matrix, vectorizer, scaler, genres_matrix = create_feature_matrix(
    df_filtered_movies)         #Matrix Maker

c:\Coding\Python-Henrik-Pedersen-OPA2023\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [114]:
def fit_knn_model(feature_matrix, n_neighbors=5):
    
    knn_model = NearestNeighbors(
        metric='cosine', algorithm='brute', n_neighbors=n_neighbors, n_jobs=-1)
    knn_model.fit(feature_matrix)
    
    return knn_model

In [216]:
def Recommender(df, movie_index, knn_model, matrix,  n_neighbors=5):
    """
    Recommends movies similar to the movie at the given index,
    including additional recommendations from the top and bottom 10% most rated movies.
    
    :param df: DataFrame containing movies and their features
    :param movie_index: Index of the movie in df to base recommendations on
    :param knn_model: Fitted KNN model
    :param genres_matrix: Genres matrix used for feature extraction
    :param top_rated_movies: DataFrame containing top 10% most rated movies
    :param bottom_rated_movies: DataFrame containing bottom 10% least rated movies
    :param n_neighbors: Number of similar movies to find
    :return: DataFrame containing recommended movies
    """
    knn_model.fit(matrix)

    # Find similar movies using KNN
    distances, indices = knn_model.kneighbors(
        matrix[movie_index], n_neighbors=n_neighbors + 1)

    recommended_movie_indices = indices.flatten()[0:] # Lets keep the original
    recommended_movies = df.iloc[recommended_movie_indices]

    return recommended_movies[["title", "genres", "bayesian_avg", "reviews"]]

In [217]:
knn_model = fit_knn_model(feature_matrix)

In [218]:
vectorizer = TfidfVectorizer(
    tokenizer=identity_tokenizer, lowercase=False, preprocessor=lambda x: x)    # Called inside the method hence white
_ = vectorizer.fit(df_filtered_movies["genres"])
vocab = vectorizer.vocabulary_
idf_scores = vectorizer.idf_
print("Vocabulary:\n", vocab)
print("\nIDF Scores:\n", idf_scores)

Vocabulary:
 {'adventure': 2, 'animation': 3, 'children': 4, 'comedy': 5, 'fantasy': 9, 'romance': 15, 'drama': 8, 'action': 1, 'crime': 6, 'thriller': 17, 'horror': 11, 'mystery': 14, 'sci-fi': 16, 'war': 18, 'musical': 13, 'documentary': 7, 'imax': 12, 'western': 19, 'film-noir': 10, '(no genres listed)': 0}

IDF Scores:
 [7.02726342 2.61478416 2.95419626 3.92669709 3.63487667 2.01908729
 3.01897376 4.41874281 1.76130614 3.47041273 5.62789698 3.23473544
 4.96106711 4.3957533  3.66360396 2.77726555 3.22882176 2.47727218
 4.19095888 5.02294251]


c:\Coding\Python-Henrik-Pedersen-OPA2023\venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [219]:
# Sum TF-IDF scores for each term across all movies
sums = genres_matrix.sum(axis=0)
# Connect term to its sum score
data = []
for col, term in enumerate(sorted(vocab.keys())):
    data.append((term, sums[0, col]))
ranking = pd.DataFrame(data, columns=['term', 'rank'])
print(ranking.sort_values('rank', ascending=False))

                  term         rank
8                drama  2115.412982
5               comedy  1750.922945
17            thriller  1000.963102
15             romance   930.161635
1               action   855.127325
6                crime   689.866266
2            adventure   616.385241
11              horror   607.301069
16              sci-fi   532.199027
9              fantasy   405.483061
14             mystery   368.721409
4             children   353.830335
3            animation   264.419484
18                 war   258.809807
7          documentary   244.700130
13             musical   197.880662
19             western   118.347540
12                imax   101.798590
10           film-noir    62.932392
0   (no genres listed)    18.000000


In [155]:
# Assuming you have these from your TF-IDF Vectorizer
idf_scores = vectorizer.idf_
genre_names = sorted(vectorizer.vocabulary_, key=vectorizer.vocabulary_.get)

# Print each genre with its corresponding IDF score
for genre, score in zip(genre_names, idf_scores):
    print(f"{genre}: {score}")

(no genres listed): 7.027263420536884
action: 2.6147841573473034
adventure: 2.954196256762068
animation: 3.9266970856673127
children: 3.6348766685568834
comedy: 2.0190872880523245
crime: 3.018973755078456
documentary: 4.418742814781707
drama: 1.7613061445985498
fantasy: 3.470412729162935
film-noir: 5.62789697784964
horror: 3.234735441701469
imax: 4.961067105607069
musical: 4.395753296557008
mystery: 3.663603958193794
romance: 2.7772655486029896
sci-fi: 3.228821763911421
thriller: 2.477272184671759
war: 4.190958883910994
western: 5.022942509325156


In [220]:
sparsity = 1.0 - feature_matrix.nnz / \
    (feature_matrix.shape[0] * genres_matrix.shape[1])

sparsity_rounded = round(sparsity, 2)
print("Sparsity of TF-IDF Matrix:", sparsity_rounded, "%")

Sparsity of TF-IDF Matrix: 0.83 %


In [224]:
recommendations = Recommender(df_filtered_movies, 325, knn_model=knn_model, matrix=feature_matrix)

In [225]:
recommendations

,title,genres,bayesian_avg,reviews
7009,Trainwreck (2015),"[comedy, romance]",3.2,2014
2974,Someone Like You (2001),"[comedy, romance]",3.2,1352
198,Forget Paris (1995),"[comedy, romance]",3.2,6825
6193,No Strings Attached (2011),"[comedy, romance]",3.2,3095
4918,Guess Who (2005),"[comedy, romance]",3.2,895
4622,"Princess Diaries 2: Royal Engagement, The (2004)","[comedy, romance]",3.2,2481
